In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [ ]:
import statsmodels.api as sm
from patsy import dmatrices

### Model Crashes <-> Project Length + Volume + Volume/Functional Class
1. Is the number of crashes associated with project length?

In [ ]:
crashes = pd.read_csv('output_2023_09_05/reports/safety-4-combined-b-crashes-all.csv')
Ljvf = pd.read_csv('output_2023_09_05/reports/overall-3-reach-Ljvf.csv')
reach = pd.read_csv('output_2023_09_05/reports/overall-2-reach-type.csv')
general = pd.read_csv('output_2023_09_05/reports/overall-1-general.csv')

First, make a new table with both the ECmoj model from the crashes table, and the associated project length/intersection count from the overall reach type

In [ ]:
crashes_reach = pd.merge(crashes, reach, on="Project ID")
crashes_reach = crashes_reach.rename(columns = {"K estimate":"K Estimate"})

In [ ]:
crashes_reach_data = crashes_reach.rename(columns = {"M Mode":"mode","O Outcome":"outcome","J Location":"location", "K Estimate":"estimate","ECmoj with user input":"ECmoj_user","ECmoj model": "ECmoj_model","Intersections count":"intersection_count","Segments count":"segment_count","Total length of segments":"segment_length"})

In [ ]:
crashes_reach_intersection = crashes_reach_data[crashes_reach_data["location"] == "intersection"]
crashes_reach_segment = crashes_reach_data[crashes_reach_data["location"] == "roadway"]

Next, make the models for segment length/intersection count only

In [ ]:
## y, X = dmatrices('ECmoj_model ~ segment_length + mode + outcome', data=crashes_reach_segment[crashes_reach_segment["Type"] == "network"], return_type = 'dataframe')
## model = sm.OLS(y,X)
## res = model.fit()
## print(res.summary())
## Idk whether to put all these variables, probably not?

In [ ]:
y, X = dmatrices('ECmoj_model ~ segment_length', data=crashes_reach_segment[crashes_reach_segment["Type"] == "project total"], return_type = 'dataframe')
model = sm.OLS(y,X)
res = model.fit()
print(res.summary())

In [ ]:
y2, X2 = dmatrices('ECmoj_model ~ intersection_count', data=crashes_reach_intersection[crashes_reach_intersection["Type"] == "project total"], return_type = 'dataframe')
model2 = sm.OLS(y2,X2)
res2 = model2.fit()
print(res2.summary())
## The conditional number for this is very low... if that is a good thing (it isn't showing the warning anymore at least)?

### Try to bring in volume

In [ ]:
volume = crashes = pd.read_csv('output_2023_09_05/reports/safety-5-volume-d-combined.csv')
volume = volume.rename(columns={"J Location type":"J Location"})

In [ ]:
## Add volume to the combined table
crashes_reach_volume = pd.merge(crashes_reach, volume, on=["Project ID","M Mode","J Location","K Estimate"])
crashes_reach_volume

### Graphs - split up the model and compare/correlate variables to the model outputs

In [ ]:
crashes_reach_volume.groupby("J Location").plot(y=["ECmoj model"])
## Intersections looks really off! Segments are actually kind of ok - only up to 30 crashes.
## What is different about segments vs intersections?

In [ ]:
crashes_reach_volume[crashes_reach_volume["J Location"] == "roadway"].plot.scatter(x=["Total length of segments"], y=["ECmoj model"])
## Also, looks like there is weird behavior when the length/count is 0?

In [ ]:
crashes_reach_volume[crashes_reach_volume["J Location"] == "intersection"].plot.scatter(x=["Intersections count"], y=["ECmoj model"])
## Also, looks like there is weird behavior when the length/count is 0?

In [ ]:
[crashes_reach_volume["Type"] == "project total"]

In [ ]:
.plot.scatter(x=["Total length of segments"], y=["ECmoj model"])

In [ ]:
crashes_reach_volume_user = crashes_reach_volume[crashes_reach_volume["Type"] == "user defined"][(crashes_reach_volume["Total length of segments"] > 0) | (crashes_reach_volume["Intersections count"] > 0)]
crashes_reach_volume_user.groupby("J Location").plot(y=["ECmoj model"])

In [ ]:
crashes_reach_volume[crashes_reach_volume["J Location"] == "roadway"].groupby(["Type"]).plot.scatter(x=["Total length of segments"], y=["ECmoj model"])
## Looks like there is some weird behavior when user defined segments have a length of 0... 
## oh nvm that must just be projects with 0 user defined, all network only

In [ ]:
crashes_reach_volume[crashes_reach_volume["J Location"] == "intersection"].groupby(["Type"]).plot.scatter(x=["Intersections count"], y=["ECmoj model"])
## Note that the ratio between intersections count and ECmoj seems to be much higher than the ratio between segment length and ECmoj

In [ ]:
crashes_reach_volume.groupby("J Location").plot.scatter(x=["Existing Volume Vmj"], y=["ECmoj model"])
## Similar situation with volume <-> crashes - for some reason, much higher for intersections than roadways
## These look maybe a bit more curved (exponential/quadratic?) than linear?

### Add volume to regressions
Now to make a regression - let's split by location type since there seems to be a big difference here

In [ ]:
crashes_reach_volume_data = crashes_reach_volume.rename(columns = {"M Mode":"mode","O Outcome":"outcome","J Location":"location", "K Estimate":"estimate","ECmoj with user input":"ECmoj_user","ECmoj model": "ECmoj_model","Intersections count":"intersection_count","Segments count":"segment_count","Total length of segments":"segment_length", "Existing Volume Vmj":"Vmj", "Projected Volume PVmjk":"Pvmjk"})

In [ ]:
crashes_reach_volume_intersection = crashes_reach_volume_data[crashes_reach_volume_data["location"] == "intersection"]
crashes_reach_volume_segment = crashes_reach_volume_data[crashes_reach_volume_data["location"] == "roadway"]

In [ ]:
y3, X3 = dmatrices('ECmoj_model ~ segment_length + Vmj', data=crashes_reach_volume_segment[crashes_reach_volume_segment["Type"] == "project total"], return_type = 'dataframe')
model3 = sm.OLS(y3,X3)
res3 = model3.fit()
print(res3.summary())

In [ ]:
sm.graphics.plot_partregress_grid(res3,['segment_length','Vmj'])

In [ ]:
## Individual graphs (to see more easily)
sm.graphics.plot_partregress('ECmoj_model','segment_length',['Vmj'], data=crashes_reach_volume_segment[crashes_reach_volume_segment["Type"] == "network"], obs_labels=False)

In [ ]:
sm.graphics.plot_partregress('ECmoj_model','Vmj',['segment_length'], data=crashes_reach_volume_segment[crashes_reach_volume_segment["Type"] == "network"], obs_labels=False)

In [ ]:
## Now for intersections
y4, X4 = dmatrices('ECmoj_model ~ intersection_count + Vmj', data=crashes_reach_volume_intersection[crashes_reach_volume_intersection["Type"] == "project total"], return_type = 'dataframe')
model4 = sm.OLS(y4,X4)
res4 = model4.fit()
print(res4.summary())
## Both in this model and the first one with intersection count, the coefficient of intersection count has been ~2.
# I wonder if there was some constant multiplier doubling intersection counts somewhere?
# I should also split by mode and retry...

In [ ]:
sm.graphics.plot_partregress_grid(res4,['intersection_count','Vmj'])

In [ ]:
## Individual graphs (to see more easily)
sm.graphics.plot_partregress('ECmoj_model','intersection_count',['Vmj'], data=crashes_reach_volume_intersection[crashes_reach_volume_intersection["Type"] == "project total"], obs_labels=False)

In [ ]:
sm.graphics.plot_partregress('ECmoj_model','Vmj',['intersection_count'], data=crashes_reach_volume_intersection[crashes_reach_volume_intersection["Type"] == "project total"], obs_labels=False)

### Try combinations of variables: length/count * Vmj

In [ ]:
# Start with intersections - since that is clearly the problem
y5, X5 = dmatrices('ECmoj_model ~ intersection_count * Vmj', data=crashes_reach_volume_intersection[crashes_reach_volume_intersection["Type"] == "project total"], return_type = 'dataframe')
model5 = sm.OLS(y5,X5)
res5 = model5.fit()
print(res5.summary())

In [ ]:
sm.graphics.plot_partregress_grid(res5)
## I have no idea what this means... need to read up about it before interpreting!